In [16]:
import os
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import date

# Download data

In [17]:
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

In [18]:
payload = {
    'client_id': "104136",
    'client_secret': 'd84d42afe9b3f6922d70c659faac1d96875d6dad',
    'refresh_token': '36a5d7075252c21122db9c00e1ef9c032411a549',
    'grant_type': "refresh_token",
    'f': 'json'
}

In [19]:
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

Requesting Token...

Access Token = 962d03610e681a13c1f874940df1f0e225e17831



In [20]:
header = {'Authorization': 'Bearer ' + access_token}
my_dataset = []

page = 1
param = {'per_page': 200, 'page': page}
data_page = requests.get(activites_url, headers=header, params=param).json()
while len(data_page)>0:
    print(page)
    param = {'per_page': 200, 'page': page}
    data_page = requests.get(activites_url, headers=header, params=param).json()
    my_dataset += data_page
    page += 1


1
2
3
4


In [21]:
print(my_dataset[0]["name"])
print(len(my_dataset))

Zurich Marató Barcelona
430


# Convert to pandas and save in csv

In [22]:
import pandas as pd
from pandas.io.json import json_normalize
activities = json_normalize(my_dataset)

C:\Users\nicol\AppData\Local\Temp\ipykernel_10176\2102315307.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  activities = json_normalize(my_dataset)


In [23]:
activities.keys()

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_temp', 'has_heartrate',
       'average_heartrate', 'max_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'suffer_score', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'average_watts', 'kilojoules', '

In [24]:
activities.head()

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_watts,kilojoules,device_watts
0,2,Zurich Marató Barcelona,42626.9,10658,10663,147.0,Run,Run,1.0,8739724557,...,False,407.0,21521933,1,a8739724557,u|o{FkybLs@~BcB`DYrB@zHOrAJpAOjBZvF]jGLxBK~PWr...,2,NaN,NaN,NaN
1,2,Lunch Run,1079.4,311,311,34.0,Run,Run,0.0,8733560415,...,False,7.0,21521933,1,a8733560415,_lg|F}~bLVh@DTQNs@hAYf@kA`BMZGJKH]v@g@l@Qb@KLW...,2,NaN,NaN,NaN
2,2,Morning Run,5838.6,1508,1514,27.0,Run,Run,0.0,8733560200,...,False,34.0,21521933,1,a8733560200,o`h|Fe}aLJIn@Un@{@jAqA^s@LOBMTg@COc@i@KUNc@Zg@...,2,NaN,NaN,NaN
3,2,Morning Run,6639.4,1826,1861,54.0,Run,Run,0.0,8727788722,...,False,41.0,21521933,1,a8727788722,umh|Fox`LLh@Bh@Kp@Af@Gf@Q`@SXQNg@\m@Z}ArAKPENA...,2,NaN,NaN,NaN
4,2,Evening Run,8258.3,2101,2146,64.0,Run,Run,NaN,8719918522,...,False,58.0,21521933,1,a8719918522,y_h|Fy|aL\ULM\g@`A}@v@{AZYFMZ]Gc@c@e@CQBS|@sA\...,2,NaN,NaN,NaN


In [27]:
activities.to_csv(os.path.join('..','data','my_dataset_%s.csv'%str(date.today())), index=False)